In [1]:
# initialize stuff
import sys
import logging
import importlib

import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt

from pathlib import Path
from tqdm import tqdm
from scipy.optimize import curve_fit

# import theory stuff

HOME_DIR = "/Users/oliver/Documents/p5control-bluefors-evaluation"
sys.path.append(HOME_DIR)

import superconductivity.api as sc

from superconductivity.style.cpd4 import cmap, colors

import numpy as np

from matplotlib.markers import MarkerStyle
from matplotlib.lines import Line2D

import superconductivity.api as sc

from superconductivity.api import G_0_muS
from superconductivity.api import NDArray64

from IPython import get_ipython

_ip = get_ipython()
if _ip is not None:
    _ip.run_line_magic("matplotlib", "inline")
    _ip.run_line_magic("reload_ext", "autoreload")
    _ip.run_line_magic("autoreload", "2")

    _ip.run_line_magic(
        "config",
        "InlineBackend.print_figure_kwargs = {'bbox_inches': None, 'pad_inches': 0.0}",
    )
    _ip.run_line_magic("config", 'InlineBackend.figure_format = "retina"')  # or "png"
    _ip.run_line_magic(
        "config", "InlineBackend.rc = {'figure.dpi': 300}"
    )  # choose a value you like

In [2]:
Delta_meV = 0.19345
G_N = 0.18877
nu_GHz = 13.6
eta = 0.002175
title_html = "TB irradiated with <i>ν = </i>13.6 GHz"
title = "TB, nu=13.6GHz"
dataset = Path("TB", "nu=13.6GHz")
expdata = "amplitude at 13.6GHz"

# Delta_meV = 0.1895
# nu_GHz = 15.0
# eta = 0.0035
# dataset = "2.09G0/nu=15GHz"
# title = "Amplitude Study (15GHz, Antenna)"

In [3]:
from utilities.ivplot import IVPlot

importlib.reload(sys.modules["utilities.ivevaluation"])
eva = IVPlot()
eva.sub_folder = ""
eva.title = expdata
eva.loadData()

(base) ... BaseClass initialized.
(base eva) ... BaseEvaluation initialized.
(iv eva) ... IVEvaluation initialized.
(base) ... BaseClass initialized.
(base plot) ... BasePlot initialized.
(iv plot) ... IVPlot initialized.
(base) amplitude at 13.6GHz
(base) loadData()


In [4]:
z = eva.up_sweep["differential_conductance"] / G_N
x = eva.mapped["voltage_axis"] * 1e3 / Delta_meV
y = eva.mapped["y_axis"] * 1e3 * eta / (sc.h_e_pVs * nu_GHz)

In [5]:
from __future__ import annotations

import numpy as np


def remap_to_regular_grid_nearest(
    src: np.ndarray,
    size_x: int,
    size_y: int,
) -> np.ndarray:
    """
    Remap a 2D array to a regular (size_y, size_x) grid using nearest-neighbor
    sampling (closest value). Oversampling naturally fills gaps by repetition.

    Parameters
    ----------
    src
        Source array of shape (NY, NX).
    size_x
        Target grid width (number of columns).
    size_y
        Target grid height (number of rows).

    Returns
    -------
    out
        Remapped array of shape (size_y, size_x).
    """
    if src.ndim != 2:
        raise ValueError(f"src must be 2D, got shape={src.shape!r}")
    if size_x <= 0 or size_y <= 0:
        raise ValueError("size_x and size_y must be positive")

    ny, nx = src.shape

    # Map target pixel centers to source pixel centers
    sx = nx / size_x
    sy = ny / size_y

    xs = (np.arange(size_x) + 0.5) * sx - 0.5
    ys = (np.arange(size_y) + 0.5) * sy - 0.5

    ix = np.rint(xs).astype(np.int64)
    iy = np.rint(ys).astype(np.int64)

    ix = np.clip(ix, 0, nx - 1)
    iy = np.clip(iy, 0, ny - 1)

    # Fancy indexing: broadcast iy (rows) against ix (cols)
    out = src[iy[:, None], ix[None, :]]
    return out

In [27]:
from luanti.dataset import export_dataset
from superconductivity.style.cpd4 import cmap as get_cmap

nu_z = remap_to_regular_grid_nearest(z, 401, 201)
export_dataset(
    nu_z,
    name="map",
    title="dIdV exp",
    zlim=(0.5, 1.5),
    hmin=25,
    hmax=76,
    colormap=get_cmap(),
)
nu_z.shape

(201, 401)

chmod +x move_mod.sh
./move_mod.sh

open -a TextEdit ~/Library/Application\ Support/minetest/debug.txt


In [28]:
from luanti.build_worlds import main

importlib.reload(sys.modules["luanti.build_worlds"])

built = main(deploy=True)

built world: my dIdV_exp
Deployed mods and worlds to:
  /Users/oliver/Library/Application Support/minetest


In [91]:
import numpy as np
from matplotlib import cm
from PIL import Image
from superconductivity.style.cpd4 import cmap as get_cmap

cmap = cm.get_cmap("viridis", 256)
cmap = get_cmap()
rgb = (cmap(np.arange(256))[:, :3] * 255).astype(np.uint8)  # (256,3)
img = Image.fromarray(rgb[np.newaxis, :, :], mode="RGB")  # 1x256
img.save("luanti/mods/measurement_terrain/cmap.png")

/var/folders/kc/8fnzl3f94vxgl8w4wm3wfvk80000gn/T/ipykernel_82569/410825988.py:6: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap("viridis", 256)
